In [66]:
# импорт библиотек
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image

import os
from os import listdir

from torchvision import transforms
from torchvision.io import read_image
import torch

In [67]:
# подготовка изображения
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25]),
])

In [68]:
def get_acc(network_architecture):

    folder_dir = "D:/Study/magistry/sem_1/computer_vision/ComputerVision/Lab3/dataset"

    top_5_acc_num = 0 
    top_1_acc_num = 0 
    count_images = 0

    for file_name in os.listdir(folder_dir):
            count_images += 1
            if (file_name.endswith(".jpg")):
                img = Image.open(folder_dir + '/' + file_name)
                top5_preds = network_architecture(img)
                real_class_img = (file_name[file_name.find('_') + 1 : file_name.find('.')])
                if real_class_img in top5_preds:
                    top_5_acc_num +=  1
                if real_class_img in top5_preds[0]:
                    top_1_acc_num += 1
                # else: print('no match')
    
    return top_5_acc_num/count_images , top_1_acc_num/count_images 

In [69]:
def print_classification(model):
  folder_dir = "D:/Study/magistry/sem_1/computer_vision/ComputerVision/Lab3/dataset"
  for image in os.listdir(folder_dir):
        if (image.endswith(".jpg")):
          input_image = Image.open(folder_dir+'/'+image)
          img = plt.imread(folder_dir+'/'+image)
          plt.figure(figsize=(10,5),dpi=80)
          model(input_image)
          plt.imshow(img)
          plt.axis('off')
          plt.show()

# AlexNet

In [70]:
from torchvision.models import alexnet, AlexNet_Weights

In [71]:
weights_alexnet = AlexNet_Weights.IMAGENET1K_V1
model = alexnet(weights_alexnet)

In [72]:
def model_alexnet(input_img):

  n = 5
  # массив для топ-5 предсказаний с вероятностями
  list_top5_preds = []

  image = preprocess(input_img).unsqueeze(0)  

  # ускорение алгоритма
  if torch.cuda.is_available():
    image = image.to('cuda')
    model.to('cuda')

  # выполнение предсказания  
  prediction = model(image).squeeze(0).softmax(0)
  class_id = torch.topk(prediction.flatten(), n).indices

  # вывод топ-5 предсказаний с вероятностями
  for i in range(n):
    top5_prob = prediction[class_id[i]].item()
    top5_categ = weights_alexnet.meta["categories"][class_id[i]]
    print(f"{top5_categ}: {100 * top5_prob:.1f}%")
    list_top5_preds.append(top5_categ)

  return list_top5_preds

In [73]:
%%time
# print_classification(model_alexnet)
top_5_acc, top_1_acc = get_acc(model_alexnet)
print(f'Top-5 accuracy: {100 * top_5_acc:.1f}% \nTop-1 accuracy: {100 * top_1_acc:.1f}%') 

African elephant: 97.6%
Indian elephant: 1.4%
tusker: 0.8%
Arabian camel: 0.1%
Chesapeake Bay retriever: 0.0%
airliner: 90.6%
wing: 9.3%
warplane: 0.1%
sombrero: 0.0%
knot: 0.0%
airliner: 99.8%
warplane: 0.1%
wing: 0.1%
airship: 0.0%
projectile: 0.0%
spoonbill: 31.7%
tank: 26.9%
missile: 16.0%
projectile: 4.7%
magpie: 2.9%
electric guitar: 25.0%
oboe: 10.5%
bow: 10.2%
flute: 6.6%
airliner: 6.3%
airliner: 86.7%
wing: 10.3%
radio telescope: 1.1%
airship: 0.7%
warplane: 0.6%
hammerhead: 31.3%
albatross: 28.4%
airliner: 12.9%
airship: 3.5%
missile: 3.2%
airliner: 80.5%
warplane: 14.3%
space shuttle: 4.2%
wing: 0.9%
missile: 0.1%
airliner: 29.0%
wing: 24.5%
alp: 18.3%
airship: 10.6%
ski: 3.9%
airliner: 77.7%
space shuttle: 21.5%
warplane: 0.2%
radio telescope: 0.1%
projectile: 0.1%
Indian elephant: 71.6%
African elephant: 16.9%
tusker: 11.5%
water buffalo: 0.1%
hippopotamus: 0.0%
airliner: 57.3%
warplane: 35.2%
aircraft carrier: 4.5%
space shuttle: 1.5%
wing: 1.0%
lemon: 73.9%
orange: 26.0%

# ResNet50

In [74]:
from torchvision.models import resnet50, ResNet50_Weights

In [75]:
weights_resnet50 = ResNet50_Weights.IMAGENET1K_V1
model = resnet50(weights=weights_resnet50)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [76]:
def model_resnet50(input_img):

  n = 5
  # массив для топ-5 предсказаний с вероятностями
  list_top5_preds = []

  image = preprocess(input_img).unsqueeze(0)  

  if torch.cuda.is_available():
    image = image.to('cuda')
    model.to('cuda')
        
  prediction = model(image).squeeze(0).softmax(0)
  class_id = torch.topk(prediction.flatten(), n).indices

  for i in range(n):
    top5_prob = prediction[class_id[i]].item()
    top5_categ = weights_resnet50.meta["categories"][class_id[i]]
    print(f"{top5_categ}: {100 * top5_prob:.1f}%")
    list_top5_preds.append(top5_categ)

  return list_top5_preds

In [77]:
%%time
# print_classification(model_resnet50)
top_5_acc, top_1_acc = get_acc(model_resnet50)
print(f'Top-5 accuracy: {100 * top_5_acc:.1f}% \nTop-1 accuracy: {100 * top_1_acc:.1f}%') 

African elephant: 81.8%
tusker: 11.4%
Indian elephant: 6.8%
Arabian camel: 0.0%
water buffalo: 0.0%
airliner: 96.1%
wing: 3.7%
warplane: 0.2%
projectile: 0.0%
missile: 0.0%
airliner: 91.6%
wing: 5.4%
warplane: 1.9%
projectile: 0.6%
missile: 0.4%
airliner: 86.5%
wing: 9.5%
warplane: 1.8%
projectile: 1.0%
missile: 0.7%
airliner: 95.0%
wing: 4.9%
warplane: 0.0%
space shuttle: 0.0%
projectile: 0.0%
airliner: 56.4%
wing: 41.0%
warplane: 1.7%
airship: 0.3%
space shuttle: 0.3%
airliner: 63.6%
wing: 10.3%
projectile: 9.3%
warplane: 8.2%
missile: 6.6%
airliner: 97.6%
wing: 1.8%
warplane: 0.4%
space shuttle: 0.1%
projectile: 0.0%
airliner: 94.7%
wing: 4.3%
space shuttle: 0.5%
warplane: 0.3%
airship: 0.2%
airliner: 99.1%
space shuttle: 0.4%
wing: 0.3%
warplane: 0.1%
airship: 0.1%
tusker: 35.4%
African elephant: 34.9%
Indian elephant: 28.4%
bison: 0.2%
Arabian camel: 0.1%
airliner: 95.5%
warplane: 2.4%
wing: 1.3%
space shuttle: 0.3%
aircraft carrier: 0.1%
lemon: 99.5%
orange: 0.5%
banana: 0.0%
pin

# VGG19

In [78]:
from torchvision.models import vgg19
from torchvision.models.vgg import VGG19_Weights

In [79]:
weights_vgg19 = VGG19_Weights.IMAGENET1K_V1
model = vgg19(weights=weights_vgg19)

In [80]:
def model_vgg19(input_img):

  n = 5
  # массив для топ-5 предсказаний с вероятностями
  list_top5_preds = []

  image = preprocess(input_img).unsqueeze(0)  

  if torch.cuda.is_available():
    image = image.to('cuda')
    model.to('cuda')
        
  prediction = model(image).squeeze(0).softmax(0)
  class_id = torch.topk(prediction.flatten(), n).indices

  for i in range(n):
    top5_prob = prediction[class_id[i]].item()
    top5_categ = weights_vgg19.meta["categories"][class_id[i]]
    print(f"{top5_categ}: {100 * top5_prob:.1f}%")
    list_top5_preds.append(top5_categ)

  return list_top5_preds

In [83]:
%%time
# print_classification(model_vgg19)
top_5_acc, top_1_acc = get_acc(model_vgg19)
print(f'Top-5 accuracy: {100 * top_5_acc:.1f}% \nTop-1 accuracy: {100 * top_1_acc:.1f}%') 

African elephant: 71.0%
Indian elephant: 15.3%
tusker: 13.8%
triceratops: 0.0%
water buffalo: 0.0%
airliner: 73.2%
wing: 25.7%
warplane: 1.2%
airship: 0.0%
missile: 0.0%
airliner: 88.2%
warplane: 4.2%
projectile: 3.1%
missile: 2.9%
wing: 1.6%
airliner: 69.4%
wing: 20.2%
warplane: 6.0%
missile: 2.0%
projectile: 1.8%
airliner: 57.2%
warplane: 18.0%
space shuttle: 10.3%
wing: 6.6%
missile: 4.0%
airliner: 55.1%
wing: 39.4%
warplane: 4.5%
space shuttle: 0.5%
trailer truck: 0.2%
warplane: 61.4%
airliner: 16.9%
missile: 11.1%
projectile: 7.1%
aircraft carrier: 1.3%
airliner: 87.0%
wing: 11.3%
warplane: 1.6%
space shuttle: 0.1%
missile: 0.0%
airliner: 90.4%
wing: 6.5%
warplane: 2.5%
space shuttle: 0.5%
projectile: 0.1%
airliner: 97.4%
space shuttle: 1.9%
wing: 0.3%
warplane: 0.3%
airship: 0.0%
African elephant: 74.8%
tusker: 18.7%
Indian elephant: 6.2%
bison: 0.1%
water buffalo: 0.0%
airliner: 97.6%
warplane: 1.5%
wing: 0.6%
space shuttle: 0.3%
aircraft carrier: 0.0%
lemon: 98.8%
orange: 1.1%
